<a href="https://colab.research.google.com/github/tejareddytadi/AI-PDF-Analyzer/blob/main/ai_pdfanalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain streamlit chromadb sentence-transformers transformers pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 685.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!pip install transformers sentence-transformers pypdf2


In [3]:
import PyPDF2
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text + " "
        return text.strip()


In [5]:
from sentence_transformers import SentenceTransformer

# Initialize Sentence Transformer Model
model_sentence = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def embed_documents(docs):
    return {doc_id: model_sentence.encode(text, show_progress_bar=True) for doc_id, text in docs.items()}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def retrieve_documents(query, doc_embeddings, docs):
    # Detect key terms in the query to focus retrieval
    keywords = ["risk", "Google", "Tesla"]
    matched_docs = [doc for doc, text in docs.items() if any(kw in text.lower() for kw in keywords)]
    if matched_docs:
        relevant_texts = " ".join(matched_docs)
    else:
        relevant_texts = " ".join(docs.values())  # Fallback if no direct matches are found
    return relevant_texts


In [7]:
def filter_relevant_content(text, keywords):
    sentences = text.split('.')
    filtered_sentences = [sentence.strip() + '.' for sentence in sentences if any(keyword.lower() in sentence.lower() for keyword in keywords)]
    return " ".join(filtered_sentences[:10])  # Limit to the first 10 relevant sentences


In [8]:
def generate_insight(prompt, context):
    # Combining the prompt and a brief, focused context
    focused_context = ". ".join([sentence for sentence in context.split('. ') if "risk" in sentence.lower()][:2])  # Take sentences that contain 'risk'
    input_text = prompt + " Context: " + focused_context
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids
    outputs = gpt_model.generate(inputs, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


new trail generate

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
def generate_insight(prompt, context):
    # Combine the prompt and the context for the model
    combined_input = prompt + " " + context

    # Encode the combined input using the tokenizer
    input_ids = tokenizer.encode(combined_input, return_tensors='pt')

    # Generate the output using the model
    outputs = gpt_model.generate(
        input_ids,
        max_length=512,  # Maximum length of the output text
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Helps prevent repetitive text
        temperature=0.7,  # Controls the randomness of the output
        top_p=0.95,  # Nucleus sampling: top p% most likely words are considered
        top_k=50  # Top-k sampling: the k most likely next words are sampled
    )

    # Decode the generated ids to text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [11]:
def process_query(query, docs, doc_embeddings):
    relevant_text = retrieve_documents(query, doc_embeddings, docs)
    filtered_context = filter_relevant_content(relevant_text, ['risk', 'Google', 'Tesla'])
    response = generate_insight(query, filtered_context)
    return response


In [12]:
docs = {
    "alphabet": extract_text_from_pdf("/content/drive/MyDrive/goog-10-k-2023.pdf"),
    "tesla": extract_text_from_pdf("/content/drive/MyDrive/tesla in.pdf"),
    "uber": extract_text_from_pdf("/content/drive/MyDrive/uber inc.pdf")}

doc_embeddings = embed_documents(docs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# **Testing**

In [ ]:
def interactive_test(docs, doc_embeddings):
    print("Enter 'quit' to stop testing.")
    while True:
        query = input("Enter your prompt: ")
        if query.lower() == 'quit':
            print("Exiting test...")
            break
        response = process_query(query, docs, doc_embeddings)
        print("Generated Insights:\n", response)

# Assuming docs and doc_embeddings are prepared and available
interactive_test(docs, doc_embeddings)


Enter 'quit' to stop testing.
Enter your prompt: What is the total revenue for Google Search?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Insights:
 What is the total revenue for Google Search? Extracted text from Alphabet's PDF Extracted text from Tesla's PDF Extracted text from Uber's PDF.

How much does Google search cost?
. Google searches for "Google" cost $1.25 billion in 2013, according to the company. That's a lot of money for a company that has spent $2.5 billion on advertising in the last year. But Google's search revenue is still a fraction of what it was in 2012. The company's revenue was $3.4 billion, or $4.6 billion per year, in 2011. It's also a bit lower than the $5.3 billion it spent in 2010. And it's still far below the cost of Google Maps, which costs $6.7 billion. (Google's Maps revenue in 2014 was about $8.8 billion.)
 (The company also spent about half of its revenue on ads, but that's not a huge amount of revenue.) Google also has a $10 billion budget for its search engine, and it has about a quarter of the revenue it had in 2009. So it doesn't have to spend a ton of time on its own.
Ente

Streamlit

In [ ]:
!pip install streamlit -q
!pip install chromadb sentence-transformers transformers PyPDF2 -q
!npm install -g localtunnel
!wget -q -O - ipv4.icanhazip.com
!streamlit run /content/appnew.py & npx localtunnel --port 8501


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.65s
34.106.216.3



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.216.3:8501

npx: installed 22 in 1.928s
your url is: https://strong-pillows-bathe.loca.lt
2024-06-29 12:32:36.666321: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 12:32:36.666427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 12:32:36.674006: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alr